In [1]:
import pandas as pd
from collections import Counter
import os
from textblob import TextBlob
import regex
import operator

# For bigrams creation
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

The metadata file on [Google Docs](https://docs.google.com/spreadsheets/d/1YXusiepersJ6_XGoUVEE0jfGh5NJs-5Rds2_l5ZbGik/edit?usp=sharing)

The collection object "Linné-Mexiko" from Etnografiska Museet is [on Carlotta](http://collections.smvk.se/carlotta-em/web/object/1460547)

A list of all the photographs is listed [on Carlotta](http://collections.smvk.se/carlotta-em/web/object/1460547/CHILDREN/9)

Here is the full search in the Database from Etnografiska Museet [in K-SAMSOK](http://www.varldskulturmuseerna.se/etnografiskamuseet/forskning-samlingar/sok-i-samlingarna1/?ksamsearchtext=mexiko+sigvald+tula&radio-group=andmatch&itemtype=samling&ksamsubmit=S%C3%B6k)

# 0. Read in the metadata

In [2]:
mexiko_test = pd.read_excel("excel-export.xls", sheetname="Mexiko")
mexiko_test.columns

Index(['Fotonummer', 'Postnr.', 'Motivord', 'Beskrivning', 'Land, foto',
       'Region, foto', 'Ort, foto', 'Etnisk grupp, avb.', 'Fotodatum',
       'Personnamn / fotograf', 'Personnamn / avbildad', 'Sökord',
       'Händelse / var närvarande vid', 'Länk'],
      dtype='object')

In [3]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text
    
mexiko_converters = {"Fotonummer":strip,"Postnr":strip,"Motivord":strip,"Beskrivning":strip,"Land, foto":strip,
                     "Region, foto":strip,"Ort, foto":strip,"Etnisk grupp, avb.":strip,"Fotodatum":strip,
                    "Personnamn / fotograf":strip, "Personnamn / avbildad":strip, "Sökord":strip,
                    "Händelse / var närvarande vid":strip, "Länk":strip}

mexiko = pd.read_excel("excel-export.xls", sheetname="Mexiko", converters=mexiko_converters)

# Inspect keywords in column "Motivord"

In [10]:
mexiko.Motivord.value_counts()[:10]

pyramid, tempel                115
pyramid                         76
kärl, lerkärl                   63
Ciudadela                       46
Ciudadela, skulptur             25
figur, skulptur, Xipe Totec     22
skulptur                        21
bil                             15
kaktus, vegetation              15
flygplan                         9
Name: Motivord, dtype: int64

In [16]:
motivord_count = Counter()

for index, string in mexiko.Motivord.iteritems():
    if pd.notnull(string):
        tokens = string.split(", ")
        tokens = [token.strip() for token in tokens]
        for token in tokens:
            motivord_count[token] += 1
motivord_count.most_common(10)

[('pyramid', 199),
 ('tempel', 131),
 ('Ciudadela', 81),
 ('skulptur', 72),
 ('kärl', 69),
 ('lerkärl', 63),
 ('figur', 23),
 ('Xipe Totec', 22),
 ('vegetation', 16),
 ('kaktus', 16)]

## Add sub-collection metadata field "description"

In [4]:
a_str = "1234.j.345432"

In [5]:
a_str.partition(".")

('1234', '.', 'j.345432')

In [4]:
sub_desc = {"a":"Teotihuacan (241) utgrävningar, fornlämningar, invånare",
"b": "Mexiko (29) utgrävningar, fornlämningar mm",
"c": "Tula (32) landskap, miljöer, invånare, fornlämningar",
"d": "Oaxaca (54) fornlämningar, invånare, miljöer mm. Nr.1 saknas",
"e": "Teopanzolco, Xochicalco (50) fornlänningar nr.51 saknas",
"f": "Yucatan, Chichen Itza fornlämningar, landskap. Två kartonger, 352 saknas",
"g": "Yucatan, Uxmal (113) fornlämningar",
"h": "Yucatan, Sayil (9) fornlämningar",
"i": "Yucatan, Kabah (59) fornlämningar",
"j": "Yucatan, Labna (90) fornlämningar, bilresa längs landsvägen",
"k": "Yucatan, Merida (42) miljöer, invånare",
"l": "Yucatan, Mona (11) invånare, miljöer",
"m": "Yucatan, Ticul (2) boskap",
"n": "Yucatan, Dzitas (5) miljöer",
"o": "Yucatan, Villa Hermosa (14) flygfoton",
"p": "Yucatan, skilda orter (25) tågresan Vera Cruz- Mexico",
"q": "Teotihuacan (156) arkeologiska föremål"}

In [5]:
mexiko["subcol_desc"] = 0
for index, row in mexiko.iterrows():
    #print(mexiko.loc[index,"subcol_desc"])
    for key in sub_desc:
        #print(type(row.Fotonummer))
        initial_numbers, left_dot, letter_plus_last_numbers = row.Fotonummer.partition(".")
        letter, dot, last_numbers = letter_plus_last_numbers.partition(".")
        #print(letter)
        if letter == key:
            #print("Match! index: {} letter:{} key:{} desc: {}".format(index, letter, key, sub_desc[key]))
            mexiko.loc[index, "subcol_desc"] = sub_desc[key] 
mexiko.subcol_desc.value_counts()        

Yucatan, Chichen Itza fornlämningar, landskap. Två kartonger, 352 saknas    376
Teotihuacan (241) utgrävningar, fornlämningar, invånare                     274
Teotihuacan (156) arkeologiska föremål                                      159
Yucatan, Uxmal (113) fornlämningar                                          113
Oaxaca (54) fornlämningar, invånare, miljöer mm. Nr.1 saknas                 92
Yucatan, Labna (90) fornlämningar, bilresa längs landsvägen                  90
Mexiko (29) utgrävningar, fornlämningar mm                                   81
Yucatan, Kabah (59) fornlämningar                                            59
Teopanzolco, Xochicalco (50) fornlänningar nr.51 saknas                      55
Yucatan, Merida (42) miljöer, invånare                                       42
Tula (32) landskap, miljöer, invånare, fornlämningar                         32
Yucatan, skilda orter (25) tågresan Vera Cruz- Mexico                        26
Yucatan, Villa Hermosa (14) flygfoton   

# Add wiki-formatted URL-link

In [21]:
for index, row in mexiko.iterrows():
    url = row["Länk"]
    first, slash, id_str = url.rpartition("/")
    new_url = "[" + url + " Fotonummer: " + id_str + "]"
    mexiko.loc[index, "wiki_url"] = new_url
mexiko

,Fotonummer,Postnr.,Motivord,Beskrivning,"Land, foto","Region, foto","Ort, foto","Etnisk grupp, avb.",Fotodatum,Personnamn / fotograf,Personnamn / avbildad,Sökord,Händelse / var närvarande vid,Länk,subcol_desc,wiki_url
0,0307.a.0001,2786726,"karta, Stockholm",Teotihuacans monumentalbyggnader utlagda på ka...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
1,0307.a.0002.a,2786708,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
2,0307.a.0002.b,2786732,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
3,0307.a.0003.a,2787074,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
4,0307.a.0003.b,2787080,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
5,0307.a.0004,2787085,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
6,0307.a.0005,2787105,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
7,0307.a.0005.a,2787090,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
8,0307.a.0005.b,2787095,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...
9,0307.a.0005.c,2787100,"pyramid, tempel",Solpyramiden.,Mexiko,NaN,Teotihuacan,NaN,1932,"Linné, Sigvald",NaN,NaN,Utgrävningar i Teotihuacan (1932),http://kulturarvsdata.se/SMVK-EM/fotografi/htm...,"Teotihuacan (241) utgrävningar, fornlämningar,...",[http://kulturarvsdata.se/SMVK-EM/fotografi/ht...


In [9]:
mexiko.loc[0,"wiki_url"]

'[http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786726 Fotonummer: 2786726]'

# Create list of badly filled out meatdata for WMMX

In [23]:
for index, row in mexiko.iter

Index(['Fotonummer', 'Postnr.', 'Motivord', 'Beskrivning', 'Land, foto',
       'Region, foto', 'Ort, foto', 'Etnisk grupp, avb.', 'Fotodatum',
       'Personnamn / fotograf', 'Personnamn / avbildad', 'Sökord',
       'Händelse / var närvarande vid', 'Länk', 'subcol_desc', 'wiki_url'],
      dtype='object')

In [63]:
file_table = ""
tot_cnt = 0
bad_cnt = 0
for index, row in mexiko.iterrows():
    tot_cnt += 1
    if pd.isnull(row["Motivord"]) and pd.notnull(row["Ort, foto"]):
        bad_cnt += 1
        file_table += "! " + str(bad_cnt) + "\n" 
        file_table += "| " + row["wiki_url"] + "\n"
        file_table += "| " + str(row["Region, foto"]) + "\n"
        file_table += "| " + str(row["Ort, foto"]) + "\n"
        file_table += "| No content keyword\n"
        file_table += "|\n"
        file_table += "|\n"
        file_table += "|-\n"
    
    elif pd.notnull(row["Motivord"]):
        motivord = row["Motivord"]
        bad_keywords = ["pyramid","tempel","Ciudadela","tempelpyramid", "tempelpyramider","ruiner","fornlämningar"]
        sep_motivord = [token.strip() for token in motivord.split(",")]
        for token in sep_motivord:
            if token in bad_keywords:
                #print("{} depicts a pyramid, a tempel or ciudadela!".format(row["Motivord"]))
                diff = set(sep_motivord) - set(bad_keywords)
                #print(diff)
                if diff == set():
                    bad_cnt += 1
                    file_table += "! " + str(bad_cnt) + "\n" 
                    file_table += "| " + row["wiki_url"] + "\n"
                    file_table += "| " + str(row["Region, foto"]) + "\n"
                    file_table += "| " + str(row["Ort, foto"]) + "\n"
                    file_table += "| Generic content keywords\n"
                    file_table += "|\n"
                    file_table += "|\n"
                    file_table += "|-\n"
file_table += "|}"

table_header = """{| class="wikitable sortable\n"""
table_header += "|+ Fotos with bad metadata (" + str(bad_cnt) + "/" + str(tot_cnt) + ") needing manual tagging\n"
table_header += """|-
! number
! file_url
! region
! place
! why_bad? 
! Commons Cat(-s separated by comma)
! Wikidata item (-s separated by comma)
|-
"""
full_table = table_header + file_table

print("== Generic keywords: ==")
for token in bad_keywords:
    print("* " + token + ", " + str(motivord_count[token]), end="\n")
print()

print(full_table)

== Generic keywords: ==
* pyramid, 199
* tempel, 131
* Ciudadela, 81
* tempelpyramid, 5
* tempelpyramider, 2
* ruiner, 11
* fornlämningar, 3

{| class="wikitable sortable
|+ Fotos with bad metadata (1140/1442) needing manual tagging
|-
! number
! file_url
! region
! place
! why_bad? 
! Commons Cat(-s separated by comma)
! Wikidata item (-s separated by comma)
|-
! 1
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786708 Fotonummer: 2786708]
| nan
| Teotihuacan
| Generic content keywords
|
|
|-
! 2
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786708 Fotonummer: 2786708]
| nan
| Teotihuacan
| Generic content keywords
|
|
|-
! 3
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786732 Fotonummer: 2786732]
| nan
| Teotihuacan
| Generic content keywords
|
|
|-
! 4
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786732 Fotonummer: 2786732]
| nan
| Teotihuacan
| Generic content keywords
|
|
|-
! 5
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2787074 Fotonummer: 2787074]

In [41]:
motivord.find("pyramid")

-1

# Create metadata dataframe to convert to wikitable

In [10]:
headers = mexiko.columns.tolist()
headers

['Fotonummer',
 'Postnr.',
 'Motivord',
 'Beskrivning',
 'Land, foto',
 'Region, foto',
 'Ort, foto',
 'Etnisk grupp, avb.',
 'Fotodatum',
 'Personnamn / fotograf',
 'Personnamn / avbildad',
 'Sökord',
 'Händelse / var närvarande vid',
 'Länk',
 'subcol_desc',
 'wiki_url']

In [11]:
en_headers = ["photo_no","post_no","content_words","desc","country","region","place",
              "ethnic_group_depict","date_of_photo","name_photographer","name_depicted",
             "search_words","event_or_attending_at","url","subcol_desc","wiki_url"]

In [12]:
table_string = ""
# Declare table
table_header = """{| class="wikitable"
|-
"""
table_string += table_header

# Column headers
for header in en_headers:
    table_string += "! " + header + "\n"
table_string += "|-\n"

iter_limit = 3 # for testing purpose
for index, row in mexiko.iterrows():
    if index < iter_limit:
        for col in headers:
            table_string += "| " + str(row[col]) + "\n"
        table_string += "|-\n"
    else:
        break
    
# Close table
table_string += "|}"
print(table_string)

{| class="wikitable"
|-
! photo_no
! post_no
! content_words
! desc
! country
! region
! place
! ethnic_group_depict
! date_of_photo
! name_photographer
! name_depicted
! search_words
! event_or_attending_at
! url
! subcol_desc
! wiki_url
|-
| 0307.a.0001
| 2786726
| karta, Stockholm
| Teotihuacans monumentalbyggnader utlagda på karta över Stockholm.
| nan
| nan
| nan
| nan
| nan
| nan
| nan
| nan
| nan
| http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786726
| Teotihuacan (241) utgrävningar, fornlämningar, invånare
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786726 Fotonummer: 2786726]
|-
| 0307.a.0002.a
| 2786708
| pyramid, tempel
| Solpyramiden.
| Mexiko
| nan
| Teotihuacan
| nan
| 1932
| Linné, Sigvald
| nan
| nan
| Utgrävningar i Teotihuacan (1932)
| http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786708
| Teotihuacan (241) utgrävningar, fornlämningar, invånare
| [http://kulturarvsdata.se/SMVK-EM/fotografi/html/2786708 Fotonummer: 2786708]
|-
| 0307.a.0002.b
| 2786732
| 

for row in mexiko.itertuples()[:3]:
    print(row)
    

# 

In [82]:
mexiko.to_csv("enriched_mexiko_metadata_table.csv", index=False)

# 1. Create category/wikidata mapping tables

In [13]:
stopwords = [w.rstrip() for w in open("./stopwords.txt").readlines()]
stopwords

['i',
 'från',
 'och',
 'på',
 'in',
 'med',
 'vid',
 'mot',
 'efter',
 '1',
 'Från',
 'utanför',
 'fråga',
 'till',
 'av',
 '8',
 '14',
 '10',
 '11',
 'No',
 'en',
 'no',
 'nan',
 'rummet',
 'C',
 '18',
 '2',
 'rum',
 'I',
 '7',
 'J',
 '17',
 '16',
 '19',
 '22',
 'under',
 'fr',
 'Före',
 'hos',
 'mellan',
 'Se',
 '15',
 'T',
 '53',
 't',
 'E',
 'före',
 '5',
 '20',
 '4',
 '317',
 '9',
 'G',
 'går',
 'sitt',
 '152',
 '24',
 '501',
 '67',
 '12',
 'h',
 '117',
 'framför',
 'Z',
 '47',
 'också',
 'M',
 '93',
 '154',
 '32',
 '290',
 'över',
 '1927',
 'allt',
 '21',
 'öppnad',
 '25',
 'allo',
 '406',
 'lagad',
 'sista',
 'VI',
 'B',
 's',
 '39',
 'sig',
 'packar',
 'klippt',
 '318',
 'sen',
 'förbi',
 '56',
 'urskiljs',
 'blir',
 '50',
 'del',
 'åt',
 '170',
 '156',
 'T77',
 'sänds',
 'man',
 'kolla',
 'källan',
 'ett',
 'framgrävande',
 'stoppar',
 'bergen',
 'första',
 'Dagen',
 'det',
 'Rör',
 'T61',
 '65',
 'drar',
 'lastat',
 'L8',
 '15899D',
 '292',
 'vänster',
 '42',
 'skick',
 'här

## Column "Motivord"

The mapping tables are pasted onto [Commons](https://commons.wikimedia.org/wiki/Commons:Medelhavsmuseet/batchUploads/Mexiko_keywords)

In [14]:
list_of_strings = mexiko.Motivord.values.astype("str")
chunk_of_strings = "" 

for string in list_of_strings:
    clean_string = regex.sub("[\"\'\.\!\?\:\(\),;]| - ","",string) # remove .'s
    chunk_of_strings += " " + clean_string

#chunk_of_strings # separated by ","

nyckelord_list = [phrase.strip() for phrase in chunk_of_strings.split(" ") if phrase not in stopwords]
#print(nyckelord_list[:10])

nyckelord_freq = Counter(nyckelord_list)
nyckelord_freq.most_common(20)

[('pyramid', 199),
 ('tempel', 131),
 ('Ciudadela', 81),
 ('skulptur', 72),
 ('kärl', 69),
 ('lerkärl', 63),
 ('figur', 23),
 ('Xipe', 22),
 ('Totec', 22),
 ('vegetation', 16),
 ('kaktus', 16),
 ('bil', 15),
 ('ruiner', 11),
 ('flygplan', 9),
 ('flygbild', 7),
 ('flygfoto', 7),
 ('utgrävning', 7),
 ('kvinna', 6),
 ('kranium', 6),
 ('tempelpyramid', 5)]

In [15]:
header = "== Keywords from column '''Motivord''' (as is, separated by comma) ==\n"
header_row = """{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Motivord
! frequency
! category
! wikidata
|-\n"""

data_rows = []
for kw, count in  nyckelord_freq.most_common(50): # original 12 stops at 4 occurances of "bad"
    nyckelord = "| " + kw + "\n"
    
    freq = "| " + str(count) + "\n"
    the_rest = "| \n| \n|-"
        
    row = nyckelord + freq + the_rest
    
    data_rows.append(row)
        
table_ending = "\n|}"
#print(data_rows)
nyckelord_wikitable = header + header_row + "\n".join(data_rows) + table_ending
print(nyckelord_wikitable)

== Keywords from column '''Motivord''' (as is, separated by comma) ==
{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Motivord
! frequency
! category
! wikidata
|-
| pyramid
| 199
| 
| 
|-
| tempel
| 131
| 
| 
|-
| Ciudadela
| 81
| 
| 
|-
| skulptur
| 72
| 
| 
|-
| kärl
| 69
| 
| 
|-
| lerkärl
| 63
| 
| 
|-
| figur
| 23
| 
| 
|-
| Xipe
| 22
| 
| 
|-
| Totec
| 22
| 
| 
|-
| vegetation
| 16
| 
| 
|-
| kaktus
| 16
| 
| 
|-
| bil
| 15
| 
| 
|-
| ruiner
| 11
| 
| 
|-
| flygplan
| 9
| 
| 
|-
| flygbild
| 7
| 
| 
|-
| flygfoto
| 7
| 
| 
|-
| utgrävning
| 7
| 
| 
|-
| kvinna
| 6
| 
| 
|-
| kranium
| 6
| 
| 
|-
| tempelpyramid
| 5
| 
| 
|-
| katedral
| 5
| 
| 
|-
| reparation
| 5
| 
| 
|-
| trappa
| 4
| 
| 
|-
| gata
| 4
| 
| 
|-
| sjö
| 4
| 
| 
|-
| porträtt
| 4
| 
| 
|-
| ruin
| 4
| 
| 
|-
| stele
| 4
| 
| 
|-
| ugn
| 3
| 
| 
|-
| fornlämningar
| 3
| 
| 
|-
| djur
| 3
| 
| 
|-
| flodfåra
| 3
| 
| 
|-
| centralpyramiden
| 3
| 
| 
|-
| åsna
| 3
| 
| 
|-
| får
| 3
| 
| 
|-
| m

## Column "Beskrivning"

In [16]:
####### Unigrams ##############
clean_tokens_list = []
mega_string = ""
list_of_strings = []
for string in mexiko.Beskrivning.values.astype("str"):
    mega_string += " " +string
    clean_string = regex.sub("[\"\”\'\.\!\?\:\,\(\);]| - "," ",string) # remove .'s
    #print(clean_string)
    tokens = clean_string.split(" ")
    clean_tokens = [word for word in tokens] # if word not in stopwords
    #print(clean_tokens)
    for token in clean_tokens:
        clean_tokens_list.append(token)
    clean_string = " ".join(clean_tokens)
    clean_ended_string = clean_string + "." # add .'s again!

    list_of_strings.append(clean_ended_string)
    
#print("clean_tokens_list:\n{}".format(clean_tokens_list))
token_count = Counter(clean_tokens_list)

####### Bigrams ##############
chunk_of_strings = "" 
chunk_of_strings += list_of_strings[0]
for string in list_of_strings[1:]:
    if string not in stopwords:
        chunk_of_strings += " " + string

### Bigrams second approach (to avoid bigrams made of end-word + first word next sentence)
token = nltk.word_tokenize(chunk_of_strings)
bigrams = ngrams(token,2)
bigrams_counter = Counter(bigrams)

clean_bigram_dict = {}
for each_tuple, freq in bigrams_counter.items():
    #print(each_tuple)
    w1, w2 = each_tuple
    forbidden_chars = set([",","."])
    if w1 in forbidden_chars or w2 in forbidden_chars:
        continue
    elif freq > 3:
        clean_bigram_dict[w1 + " " + w2] = freq
    else:
        continue
#print(type(bigrams_counter))
#print(list(bigrams_counter)[:2])
sorted_clean_bigram_dict = sorted(clean_bigram_dict.items(), key=operator.itemgetter(1), reverse=True)
print("Token count:\n{}".format(token_count.most_common(1000)))
print()
print("Bigrams:\n{}".format(sorted_clean_bigram_dict))
#for bigram in sorted_clean_bigram_dict:
#    print(bigram)

# single words

Token count:
[('', 1013), ('nan', 800), ('från', 175), ('av', 168), ('Teotihuacan', 162), ('Arkeologiskt', 159), ('föremål', 159), ('Parti', 149), ('Från', 138), ('vid', 113), ('utgrävningarna', 105), ('Xolalpan', 95), ('Ciudadela', 88), ('tempel', 56), ('Krigarnas', 46), ('Cuicuilco-pyramiden', 45), ('Solpyramiden', 35), ('den', 35), ('katalogkort', 31), ('Dr', 30), ('El', 29), ('pyramiden', 29), ('Castillo', 27), ('centrala', 27), ('till', 26), ('i', 25), ('S', 25), ('1932', 25), ('Tenayuca', 24), ('tempelpyramiden', 24), ('Mexiko', 23), ('expedition', 22), ('Linnés', 22), ('och', 21), ('mellan', 13), ('som', 13), ('Museo', 12), ('Ticul', 12), ('Kabah', 12), ('Landsvägen', 12), ('Föremål', 12), ('Labna', 12), ('Historico', 12), ('e', 12), ('Sayil', 12), ('hus', 12), ('Arqueologico', 12), ('Merida', 12), ('Linné', 11), ('T', 11), ('på', 10), ('Thomas', 10), ('de', 10), ('Palmas', 10), ('krigarens', 9), ('v', 9), ('står', 8), ('San', 8), ('De', 7), ('Maya-indian', 7), ('Kaktusvegetatio

In [17]:
header = "== Keywords from column '''Beskrivning''' (två-ordskombinationer) ==\n"
header_row = """{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Två-ordskombination
! frequency
! category
! wikidata
|-\n"""

data_rows = []
for kw, count in sorted_clean_bigram_dict: 
    w1, w2 = kw.split()
    #print("w1: {} w2: {}".format(w1,w2))
    if w1 in stopwords or w2 in stopwords:
        print("Forbidden bigram: {}".format(kw))
    else:
        nyckelord = "| " + kw + "\n"
    
        freq = "| " + str(count) + "\n"
        the_rest = "| \n| \n|-"
        
        row = nyckelord + freq + the_rest
    
        data_rows.append(row)
        
table_ending = "\n|}"
#print(data_rows)
nyckelord_wikitable = header + header_row + "\n".join(data_rows) + table_ending
print()
print(nyckelord_wikitable)

Forbidden bigram: från Teotihuacan
Forbidden bigram: föremål från
Forbidden bigram: Parti av
Forbidden bigram: utgrävningarna vid
Forbidden bigram: Från utgrävningarna
Forbidden bigram: vid Xolalpan
Forbidden bigram: av Ciudadela
Forbidden bigram: av Cuicuilco-pyramiden
Forbidden bigram: den centrala
Forbidden bigram: Castillo den
Forbidden bigram: Från Dr
Forbidden bigram: av tempelpyramiden
Forbidden bigram: expedition till
Forbidden bigram: S Linnés
Forbidden bigram: till Mexiko
Forbidden bigram: Dr S
Forbidden bigram: från Museo
Forbidden bigram: Föremål från
Forbidden bigram: Arqueologico e
Forbidden bigram: mellan Ticul
Forbidden bigram: och Kabah
Forbidden bigram: Ticul och
Forbidden bigram: Landsvägen mellan
Forbidden bigram: e Historico
Forbidden bigram: vid Thomas
Forbidden bigram: pyramiden T
Forbidden bigram: T v
Forbidden bigram: v krigarens
Forbidden bigram: av den
Forbidden bigram: Partier av
Forbidden bigram: fasa i
Forbidden bigram: i Le
Forbidden bigram: den äldre

==

In [18]:
header = "== Keywords from column '''Beskrivning''' (word-by-word) ==\n"
header_row = """{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Ord
! frequency
! category
! wikidata
|-\n"""

data_rows = []
for kw, count in token_count.most_common(500): # original 50 stops at 3 occurances of "H"
    if kw in stopwords:
        print("Forbidden unigram: {}".format(kw))
    elif count >= 3:
        nyckelord = "| " + kw + "\n"
    
        freq = "| " + str(count) + "\n"
        the_rest = "| \n| \n|-"
        
        row = nyckelord + freq + the_rest
    
        data_rows.append(row)
    
table_ending = "\n|}"
#print(data_rows)
nyckelord_wikitable = header + header_row + "\n".join(data_rows) + table_ending
print()
print(nyckelord_wikitable)

Forbidden unigram: 
Forbidden unigram: nan
Forbidden unigram: från
Forbidden unigram: av
Forbidden unigram: Från
Forbidden unigram: vid
Forbidden unigram: den
Forbidden unigram: till
Forbidden unigram: i
Forbidden unigram: S
Forbidden unigram: och
Forbidden unigram: mellan
Forbidden unigram: som
Forbidden unigram: e
Forbidden unigram: T
Forbidden unigram: på
Forbidden unigram: de
Forbidden unigram: v
Forbidden unigram: De
Forbidden unigram: Den
Forbidden unigram: en
Forbidden unigram: ett
Forbidden unigram: med
Forbidden unigram: m
Forbidden unigram: samt
Forbidden unigram: På
Forbidden unigram: har
Forbidden unigram: delar
Forbidden unigram: vänster
Forbidden unigram: I
Forbidden unigram: äro
Forbidden unigram: var
Forbidden unigram: under
Forbidden unigram: toppen
Forbidden unigram: utanför
Forbidden unigram: för
Forbidden unigram: 12
Forbidden unigram: genom
Forbidden unigram: En
Forbidden unigram: väggen
Forbidden unigram: s
Forbidden unigram: 25
Forbidden unigram: det
Forbidden un

# Column "Sökord"

In [19]:
mexiko.Sökord.value_counts()

arkeologi, undersökning, utgrävning                                     92
arkeologi                                                               78
byggnad, fornlämning                                                    66
arkeologi, keramik                                                      63
tempelpyramid                                                           48
stenskulptur                                                            46
Altepetl Tolteca, Toltec empire, Toltec Kingdom                         21
arkeologi, lerfigur                                                     15
landskap, natur                                                          7
Altepetl Tolteca, tempelpyramid, Toltec empire, Toltec Kingdom           6
arkeologi, ben, lerfigur                                                 4
Altepetl Tolteca, stenskulptur, Toltec empire, Toltec Kingdom            4
skelett                                                                  3
arkeologi, huvud, lerfigu

In [120]:
sokord_count = Counter()

for index, value in mexiko.Sökord.iteritems():
    tokens = str(value).split(",")
    clean_tokens = [token.strip() for token in tokens]
    for token in clean_tokens:
        sokord_count[token] += 1
sokord_count.most_common()

[('nan', 953),
 ('arkeologi', 259),
 ('undersökning', 93),
 ('utgrävning', 93),
 ('fornlämning', 69),
 ('byggnad', 69),
 ('keramik', 65),
 ('tempelpyramid', 54),
 ('stenskulptur', 52),
 ('Toltec Kingdom', 38),
 ('Toltec empire', 38),
 ('Altepetl Tolteca', 38),
 ('lerfigur', 22),
 ('landskap', 7),
 ('natur', 7),
 ('kyrka', 5),
 ('skelett', 5),
 ('ben', 4),
 ('krukor', 3),
 ('kruka', 3),
 ('tegeltillverkning', 3),
 ('vatten', 3),
 ('tegel', 3),
 ('huvud', 3),
 ('byggmaterial', 3),
 ('flod', 2),
 ('mänskliga kvarlevor', 2),
 ('torg', 2),
 ('stenorm', 2),
 ('orm', 2),
 ('hantverk', 2),
 ('pensel', 2),
 ('skulptur', 2),
 ('döskalle', 2),
 ('försäljning', 2),
 ('reklam', 1),
 ('pyramid', 1),
 ('lerkärl', 1),
 ('stenfigur', 1),
 ('ruiner', 1),
 ('tempel', 1),
 ('ruin', 1),
 ('torka', 1),
 ('soldat', 1),
 ('stig', 1),
 ('kvinna', 1),
 ('fält', 1),
 ('perrong', 1),
 ('jordbruk', 1)]

In [ ]:
# unigram "Motivord"
nyckelord_freq.most_common(50)
# unigram "Beskrivning"
token_count.most_common(1000)
# bigram "Beskrivning"
sorted_clean_bigram_dict
# uni- and bigrams "Sökord"
sokord_count.most_common()

# Create mapping list for places - columns "Ort, foto", "Region, foto"

In [29]:
res = "Estado de Oaxaca, Oaxaca".partition(",")
res

('Estado de Oaxaca', ',', ' Oaxaca')

In [38]:
res = "Villahermosa".split(",")
res[-1]

'Villahermosa'

In [40]:
res = "Chichén Itzá, Dzitas".split(",")
res[-1].strip()

'Dzitas'

In [55]:
place_counter = Counter()
for index, row in mexiko.iterrows():
    if not pd.isnull(row["Ort, foto"]):
        place = row["Ort, foto"].split(",")[-1].strip()
        #print("full: {}\n place: {}\n".format(row["Ort, foto"], place))
        place_counter[place] += 1
    elif pd.isnull(row["Ort, foto"]) and pd.notnull(row["Region, foto"]): # city/village/place not specified, but
        place = row["Region, foto"].split(",")[-1].strip()
        #print("full: {}\n place: {}\n".format(row["Region, foto"], place))
        place_counter[place] += 1
    else: # neither place nor region is specified
        place_counter["n/a"] += 1
        #print("No region or place! Fotonummer: {}".format(row["Fotonummer"]))
place_counter.most_common()

[('Chichén Itzá', 376),
 ('Teotihuacan', 272),
 ('n/a', 234),
 ('Uxmal', 113),
 ('Labna', 69),
 ('Kabah', 60),
 ('Yucatan', 51),
 ('Merida', 42),
 ('Mitla', 38),
 ('Xochicalco', 37),
 ('Oaxaca', 36),
 ('Tula', 32),
 ('Teopanzolco', 18),
 ('Monte Albán', 17),
 ('Villahermosa', 16),
 ('Sayil', 9),
 ('Veracruz', 5),
 ('Tenayuca', 5),
 ('Dzitas', 5),
 ('Ticul', 2),
 ('Xlapak', 2),
 ('Mexico Distrito Federal', 1),
 ('Monte Alban', 1),
 ('Coyoacan', 1)]

In [61]:
place_table = ""
place_table += """{| class="wikitable"
|-
! Place
! Frequency
! Commons cat
! Wikidata item
|-
"""

for place, freq in place_counter.most_common():
    #print(place, freq)
    place_table += "| " + str(place) + "\n| " + str(freq) + "\n|\n|\n|-\n" 

place_table += "|}"
print(place_table)


{| class="wikitable"
|-
! Place
! Frequency
! Commons cat
! Wikidata item
|-
| Chichén Itzá
| 376
|
|
|-
| Teotihuacan
| 272
|
|
|-
| n/a
| 234
|
|
|-
| Uxmal
| 113
|
|
|-
| Labna
| 69
|
|
|-
| Kabah
| 60
|
|
|-
| Yucatan
| 51
|
|
|-
| Merida
| 42
|
|
|-
| Mitla
| 38
|
|
|-
| Xochicalco
| 37
|
|
|-
| Oaxaca
| 36
|
|
|-
| Tula
| 32
|
|
|-
| Teopanzolco
| 18
|
|
|-
| Monte Albán
| 17
|
|
|-
| Villahermosa
| 16
|
|
|-
| Sayil
| 9
|
|
|-
| Veracruz
| 5
|
|
|-
| Tenayuca
| 5
|
|
|-
| Dzitas
| 5
|
|
|-
| Ticul
| 2
|
|
|-
| Xlapak
| 2
|
|
|-
| Mexico Distrito Federal
| 1
|
|
|-
| Monte Alban
| 1
|
|
|-
| Coyoacan
| 1
|
|
|-
|}


# 2. Create new filenames for the images

In [10]:
def save_filename_to_filename_file(filname_file, filename):
    """Create a file mapping original filenames and their folders with new
    Commons filenames according to <Task X on Phabricator>"""
    folder = row["Folder"]
    file = row["Filename"]
    # Filename: <Filename_1_clean>_-_DecArch_-_<Folder_#>-<Filename_0_clean>.<ext>
    
    #print("filename: {}".format(filename))
    filenames_file.write("{}|{}|{}\n".format(row["Folder"],row["Filename"],filename))

In [11]:
def get_foldernames_and_filenames(inpath):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-11-04e94e1c82c0>, line 2)

In [ ]:
def create_filenames(fold, foldobj):
    

In [ ]:
filenames_file = open("./filenames_mapping.csv","w")
filenames_file.write("Folder|Original|Commons\n")

for row_index, row in cypern.iterrows():
    filename = create_filename(row["Folder"], row["Filename"])
    save_filename_to_filename_file(filenames_file, filename)

Infobox mapping is available on [Phabricator](https://phabricator.wikimedia.org/T144485)

In [ ]:
filenames_dict = {}
for index, row in cypern.iterrows():
    